In [1]:
import mobile_traffic as mt
from datetime import time
import noise
import insee
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
from Sleep.SessionDistribution import calculate_session_distribution_city

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
path = '/Users/anmusso/Desktop/PhD/Projects/Current/NetMob/NetMobData/data/FigureData/Sleep'

# Mobile Traffic Data

### Bed time index aggregate service

In [66]:
mobile_traffic_paris_users = mt.get_city_traffic_data(traffic_type=mt.TrafficType.USERS, city=mt.City.PARIS, geo_data_type=mt.GeoDataType.TILE)

100%|██████████| 77/77 [00:30<00:00,  2.50it/s]


In [67]:
session_distribution_paris = calculate_session_distribution_city(mobile_traffic_paris_users, start=time(22), end=time(4), window_smoothing=3)

In [68]:
bed_time_index = session_distribution_paris.get_deviation_from_mean_of_session_cumulative_distribution_by_location()
bed_time_index.reset_index(names='tile', inplace=True)
bed_time_index.rename(columns={'deviation_from_mean': 'bed_time_index'}, inplace=True)

In [69]:
bed_time_index.head()

,tile,expected_time
0,011050000,23:00:00
1,012380000,23:00:00
2,012480000,23:00:00
3,012490101,23:00:00
4,012490103,23:00:00


In [70]:
bed_time_index.to_csv(f'{path}/bed_time_index_netmob_tile.csv', index=False)

### Bed time index per service

In [ ]:
mobile_traffic_paris_ul_dl = mt.get_city_traffic_data(traffic_type=mt.TrafficType.UL_AND_DL, city=mt.City.PARIS, geo_data_type=mt.GeoDataType.TILE)

In [ ]:
bed_time_indices =[]
for service in mt.Service:
    session_distribution_paris_service = calculate_session_distribution_city(mobile_traffic_paris_ul_dl, start=time(22), end=time(4), window_smoothing=3, services=[service])
    bed_time_index_service = session_distribution_paris_service.get_deviation_from_mean_of_session_cumulative_distribution_by_location()
    bed_time_index_service.rename(columns={'deviation_from_mean': f'{service}'}, inplace=True)
    bed_time_indices.append(bed_time_index_service)

In [ ]:
bed_time_indices = pd.concat(bed_time_indices, axis=1)

In [ ]:
bed_time_indices.reset_index(names='tile', inplace=True)
bed_time_indices.head()

In [ ]:
bed_time_indices.to_csv(f'{path}/bed_time_index_netmob_tile_by_app.csv', index=False)

### Raw consumption per service

In [ ]:
service_consumption_by_location = mobile_traffic_paris_ul_dl.get_service_consumption_by_location(start=time(22), end=time(4))
service_consumption_by_location.head()

In [ ]:
service_consumption_by_location.to_csv(f'{path}/app_consumption_netmob_tile.csv', index=False)

# Noise Data

In [5]:
paris_geo = mt.geo_tile.get_geo_data(city=mt.City.PARIS)
paris_geo = paris_geo.iloc[:100]

In [62]:
noise_estimates = noise.get_noise_estimate(polygons=paris_geo, city=noise.City.PARIS, measurement=noise.Measurement.NIGHT)

In [63]:
noise_estimates = noise_estimates['noise_estimate'].to_frame()
noise_estimates.reset_index(names='tile', inplace=True)

In [64]:
noise_estimates.head()

,tile,noise_estimate
0,162,61.0
1,507,49.0
2,508,61.0
3,853,52.0
4,854,60.0


In [65]:
noise_estimates.to_csv(f'{path}/noise_estimates_paris_n.csv', index=False)

# Admin Data

In [9]:
net_mob_paris_geo = mt.geo_tile.get_geo_data(city=mt.City.PARIS)
net_mob_paris_geo.to_crs(epsg=2154, inplace=True)

In [10]:
paris_blob = net_mob_paris_geo.unary_union

In [13]:
insee_tile_geo = insee.tile.get_data(var_name='geometry')
insee_tile_geo.to_crs(epsg=2154, inplace=True)

In [15]:
insee_tile_geo = insee_tile_geo.loc[insee_tile_geo.intersects(paris_blob)]

In [16]:
equipment_counts = insee.equipment.get_equipment_counts(polygons=insee_tile_geo, resolution=insee.equipment.EquipmentResolution.HIGH)

In [17]:
codes_businesses_likely_to_be_open_at_night = ['A101', 'A104', 'A504', 'B101', 'B201', 'B316', 'C701', 'D101', 'D102', 'D103', 'D104', 'D105', 'D106', 'D107', 'D111', 'D303', 'E107', 'F303', 'G102']
codes_businesses_likely_to_be_open_at_night = ['A504', 'F303', 'G102', 'C701']

In [18]:
night_business = np.intersect1d(equipment_counts.columns, codes_businesses_likely_to_be_open_at_night)
night_equipment_counts = equipment_counts[night_business].sum(axis=1).to_frame('night_equipment_counts')

In [19]:
description_night_equip = insee.equipment.get_equipment_description(resolution=insee.equipment.EquipmentResolution.HIGH)
description_night_equip = description_night_equip.loc[description_night_equip['COD_MOD'].isin(night_business)]

In [20]:
income_and_population = insee.tile.get_data(tile=list(night_equipment_counts.index.unique()), var_name=['Ind','Ind_snv']).rename(columns={'Ind': 'pop', 'Ind_snv':'income'})
income_and_population['income'] = income_and_population['income'].div(income_and_population['pop'], axis=0)

In [21]:
age_vars =["Ind_0_3", "Ind_4_5", "Ind_6_10", "Ind_11_17", "Ind_18_24", "Ind_25_39",
    "Ind_40_54", "Ind_55_64", "Ind_65_79", "Ind_80p"]
age = insee.tile.get_data(tile=list(night_equipment_counts.index.unique()), var_name=age_vars)
age = age.div(income_and_population['pop'], axis=0)
age = age.dot([1.5, 4.5, 8, 14, 21, 32, 47, 60, 72, 85]).to_frame(name='age')

In [22]:
equip_income_pop_data = pd.merge(night_equipment_counts, income_and_population, left_index=True, right_index=True)
equip_income_pop_data_age = pd.merge(equip_income_pop_data, age, left_index=True, right_index=True)

In [23]:
equip_income_pop_data_age.reset_index(names='tile', inplace=True)

In [24]:
equip_income_pop_data_age.to_csv(f'{path}/admin_data_paris.csv', index=False)

In [25]:
equip_income_pop_data_age.head()

,tile,night_equipment_counts,pop,income,age
0,CRS3035RES200mN2866000E3759200,0.0,16.0,22193.906250,38.637500
1,CRS3035RES200mN2866000E3759400,0.0,206.0,21523.843689,34.786408
2,CRS3035RES200mN2866200E3759200,0.0,139.0,23750.458993,39.089928
3,CRS3035RES200mN2866200E3759400,1.0,352.0,16225.457670,33.355824
4,CRS3035RES200mN2866200E3759600,0.0,498.5,14109.849950,29.299398


# Matching NetMob Tile - Insee Tile

In [76]:
paris_geo = mt.geo_tile.get_geo_data(city=mt.City.PARIS)

In [6]:
matching_tile_mobile_traffic_tile_insee = insee.tile.get_matching_tiles(polygons=paris_geo, return_intersection_area=True)

In [7]:
matching_tile_mobile_traffic_tile_insee.head()

,geometry,tile,intersection_area
tile,,,
162,"POLYGON ((653218.968 6838547.550, 653219.807 6...",CRS3035RES200mN2866000E3759200,6813.063953
162,"POLYGON ((653218.968 6838547.550, 653219.807 6...",CRS3035RES200mN2866000E3759400,3173.721549
507,"POLYGON ((653119.877 6838648.320, 653120.716 6...",CRS3035RES200mN2866000E3759200,8319.845685
507,"POLYGON ((653119.877 6838648.320, 653120.716 6...",CRS3035RES200mN2866200E3759200,1666.941405
508,"POLYGON ((653219.807 6838647.481, 653220.646 6...",CRS3035RES200mN2866000E3759200,5331.103238


In [8]:
matching_tile_mobile_traffic_tile_insee.rename(columns={'tile': 'tile_insee'}, inplace=True)
matching_tile_mobile_traffic_tile_insee.reset_index(names='tile_mobile_traffic', inplace=True)
matching_tile_mobile_traffic_tile_insee = matching_tile_mobile_traffic_tile_insee[['tile_mobile_traffic', 'tile_insee', 'intersection_area']]

In [11]:
matching_tile_mobile_traffic_tile_insee.to_csv(f'{path}/matching_tile_mobile_traffic_tile_insee_paris.csv', index=False)